In [1]:
import pandas as pd
import ast
import os
import traceback
import paramiko

# import from ../utils.py
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from utils import dwl_pdb_file,  get_resolution, get_dbref_data, get_gene_name, calculate_sbna_and_download

c:\Users\liyoa\anaconda3\envs\sbna\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [27]:
genes = pd.read_csv("../data_prep/Census_all_with_pdb.csv")
genes = genes[genes["Gene Symbol"].isin(["KRAS", "HRAS"])]

In [16]:
# count number of structures for all gene
n_total = genes['n_structures'].sum()

for _, row in genes.iterrows():
    gene = row['Gene Symbol']
    pdb_structures = ast.literal_eval(row['PDB Structures'])
    for pdb_id in pdb_structures:
        n_total -= 1
        print(f"Downloading PDB file {pdb_id} for {gene} ({n_total} structures remaining)", end='\r')
        # check if the file already exists
        if not os.path.exists(f'pdb_files/{pdb_id}.pdb'):
            if not dwl_pdb_file(pdb_id): # returns True or False if the file was downloaded
                continue

Failed to download PDB file 7Q9U. Status code: 404remaining))


In [28]:
genes

,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),...,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms,PDB Structures,n_structures
318,HRAS,v-Ha-ras Harvey rat sarcoma viral oncogene hom...,3265.0,11:532243-535550,1,Yes,11p15.5,yes,yes,"infrequent sarcomas, rare other tumour types",...,"E, L, M",Dom,oncogene,Mis,NaN,NaN,NaN,"CCDS7698.1,ENSG00000174775.16,HRAS1,NM_005343....","['121P', '1AA9', '1AGP', '1BKD', '1CLU', '1CRP...",228
366,KRAS,v-Ki-ras2 Kirsten rat sarcoma 2 viral oncogene...,3845.0,12:25209431-25250803,1,Yes,12p12.1,yes,NaN,"pancreatic, colorectal, lung, thyroid, AML, ot...",...,"L, E, M, O",Dom,oncogene,Mis,NaN,NaN,NaN,"CCDS8703.1,ENSG00000133703.11,KRAS1,KRAS2,NM_0...","['1D8D', '1D8E', '1KZO', '1KZP', '1N4P', '1N4Q...",295


In [29]:
pdb_data = []
uniprot_map = {}

# count number of structures for all gene
n_total = genes['n_structures'].sum()

for _, row in genes.iterrows():
    gene = row['Gene Symbol']
    pdb_structures = ast.literal_eval(row['PDB Structures'])
    for pdb_id in pdb_structures:
        n_total -= 1
        # check if the file already exists
        if not os.path.exists(f'../pdb_files/{pdb_id}.pdb'):
            continue
        # read the file
        resolution = get_resolution(pdb_id)
        dbref = get_dbref_data(pdb_id)
        for i in dbref:
            chain = i['chain']
            start = i['start']
            end = i['end']
            uniprot = i['uniprot']
            if uniprot not in uniprot_map:
                uniprot_map[uniprot] = get_gene_name(uniprot)
            assoc_gene = uniprot_map[uniprot]
            pdb_data.append([gene, row["Name"], pdb_id, resolution, chain, start, end, uniprot, assoc_gene])

pdb_data = pd.DataFrame(pdb_data, columns=['gene_symbol', 'gene_name', 'pdb_id', 'resolution', 'chain', 'seq_start', 'seq_end', 'uniprot_id', 'assoc_gene'])
pdb_data.head()
pdb_data['length'] = pdb_data['seq_end'] - pdb_data['seq_start'] + 1

In [31]:
# sort gene ascending, length descending and resolution ascneding
pdb_data.sort_values(by=['gene_symbol', 'length', 'resolution', "pdb_id"], ascending=[True, False, True, True]).to_excel("best_pdbs_kras_hras.xlsx", index=False)

Chosen structures: 
* HRAS	1IOZ A
* KRAS	7VVB A
* TP53  1TSR A, B, C
* EGFR  4UIP A 

In [3]:
try:
    calculate_sbna_and_download({"7T4J": ["A"]})
except Exception as e:
    print(f"Failed to process {pdb_id} with error:")
    print(traceback.print_exc())


R version 4.0.5 (2021-03-31) -- "Shake and Throw"
Copyright (C) 2021 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under the terms of the
GNU General Public License versions 2 or 3.
For more information about these matters see
https://www.gnu.org/licenses/.




Processing 7T4J with chain A...
Starting phenix.pdbtools
on Sun Apr 28 21:25:29 2024 by yliy0004

Processing files:
-------------------------------------------------------------------------------

  Found model, 7T4J.pdb

Processing PHIL parameters:
-------------------------------------------------------------------------------

  Adding command-line PHIL:
  -------------------------
    keep=Chain A

Final processed PHIL parameters:
-------------------------------------------------------------------------------
  data_manager {
    model {
      file = "7T4J.pdb"
    }
    default_model = "7T4J.pdb"
